### jupyter notebook venv kernel install


In [10]:
### Create virtual environment for use
# !virtualenv\Scripts\activate
# !cd toolformer
# !pip install -r requirements.txt

### Install virtual environment kernel in your jupyter notebook
# !pip install jupyter
# !pip install ipykernel
# !python -m ipykernel install --user --name virtualenv --display-name "virtualenv"

# !deactivate

###  `OPENAI_API_KEY` as envrionment variable

In [11]:
# 환경변수 확인 (Windows)
# !set

# 환경변수로 저장
# !set OPENAI_API_KEY = ""

# 환경변수 확인
# !echo %OPENAI_API_KEY%

In [ ]:
import os
os.environ.get("OPENAI_API_KEY")

In [13]:
from openai import OpenAI
client = OpenAI(api_key = os.environ.get("OPENAI_API_KEY"))

def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], 
model=model).data[0].embedding

In [19]:
# view example
embedded_text = get_embedding("I love to be vectorized")
print(len(embedded_text) == 1536)

True


# Capabilities


## Chat Completion

In [23]:
response = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angels Dodgers won the world series in 2020."},
        {"role": "user", "content" : "Where was it played?"}
    ]
)

In [43]:
response.choices[0].message

ChatCompletionMessage(content='The 2020 World Series was played at Globe Life Field in Arlington, Texas.', role='assistant', function_call=None, tool_calls=None)

## Function Calling

- How to generate function arguments

- How to call functions with model generated arguments

### How to generate function arguments


In [34]:
!pip install scipy --quiet
!pip install tenacity --quiet
!pip install tiktoken --quiet
!pip install termcolor --quiet
!pip install openai --quiet


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [46]:
import json
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored

GPT_MODEL = "gpt-3.5-turbo-0613"

In [48]:
@retry(wait = wait_random_exponential(multiplier = 1, max = 40), stop = stop_after_attempt(3))
def chat_completion_request(messages, tools = None, tool_choice = None, model = GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model = model,
            messages = messages,
            tools = tools,
            tool_choice = tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception : {e}")
        return e

In [49]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system" : "red",
        "user" : "green",
        "assistant" : "blue",
        "function" : "magenta",
    }
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))

In [50]:
# Pass function specification to Chat Completions API
# in order to generate function arguments
# that adhere to the specification on (Hypothetical) Weather API.
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_n_day_weather_forecast",
            "description": "Get an N-day weather forecast",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                    "num_days": {
                        "type": "integer",
                        "description": "The number of days to forecast",
                    }
                },
                "required": ["location", "format", "num_days"]
            },
        }
    },
]

In [51]:
messages = []
messages.append({"role": "system", "content":"Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "What's the weather like today"})
chat_response = chat_completion_request(
    messages, tools = tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message

ChatCompletionMessage(content='Sure, could you please provide me with your location?', role='assistant', function_call=None, tool_calls=None)

In [53]:
# Once we provide the missing information, it generates appropriate function arguments.
messages.append({"role": "user", "content": "I'm in Glasgow, Scotland."})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_nftqXRcns2SErK9uulvDxGjW', function=Function(arguments='{\n  "location": "Glasgow, Scotland",\n  "format": "celsius"\n}', name='get_current_weather'), type='function')])

In [65]:
# Generated Function Arguments
assistant_message.tool_calls[0].function

Function(arguments='{\n  "location": "Glasgow, Scotland",\n  "format": "celsius"\n}', name='get_current_weather')

In [66]:
# Another example using another tool invocation
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "what is the weather going to be like in Glasgow, Scotland over the next x days"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message

ChatCompletionMessage(content="Please specify the value of 'x' to get the weather forecast for that many days in Glasgow, Scotland.", role='assistant', function_call=None, tool_calls=None)

In [67]:
messages.append({"role": "user", "content": "5 days"})
chat_response = chat_completion_request(
    messages, tools=tools
)
chat_response.choices[0]

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_35gOqlcnFWz4BGqiGJl5UKAL', function=Function(arguments='{\n  "location": "Glasgow, Scotland",\n  "format": "celsius",\n  "num_days": 5\n}', name='get_n_day_weather_forecast'), type='function')]))

In [72]:
### We can FORCE specific functions, or NO USE of functions
# FORCE `get_n_day_weather_forecast`
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, tools= tools, tool_choice = {"type": "function", "function": {"name": "get_n_day_weather_forecast"}}
)
chat_response.choices[0].message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_8CE0FrTDWwsySTJzQnILV3WU', function=Function(arguments='{\n  "location": "Toronto, Canada",\n  "format": "",\n  "num_days": 1\n}', name='get_n_day_weather_forecast'), type='function')])

In [73]:
# FORCE to NOT USE function 
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me the current weather (use Celcius) for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, tools=tools, tool_choice="none"
)
chat_response.choices[0].message

ChatCompletionMessage(content='{ "location": "Toronto, Canada", "format": "celsius" }', role='assistant', function_call=None, tool_calls=None)

새로운 모델 `gpt-3.5-turbo-1106` 같은 경우에는 multiple function 을 한 턴에 calling 할 수 있게 구현해놓음

In [74]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "what is the weather going to be like in San Francisco and Glasgow over the next 4 days"})
chat_response = chat_completion_request(
    messages, tools=tools, model='gpt-3.5-turbo-1106'
)

assistant_message = chat_response.choices[0].message.tool_calls
assistant_message

[ChatCompletionMessageToolCall(id='call_iMhFNeM4QQD4b7j5OcBmVGBZ', function=Function(arguments='{"location": "San Francisco", "format": "celsius", "num_days": 4}', name='get_n_day_weather_forecast'), type='function'),
 ChatCompletionMessageToolCall(id='call_NblKm7authlTJH880FAiuQ2y', function=Function(arguments='{"location": "Glasgow", "format": "celsius", "num_days": 4}', name='get_n_day_weather_forecast'), type='function')]

### How to call functions with "model generated arguments"

how to execute functions,

when INPUTS ARE MODEL GENERATED

In [83]:
import sqlite3

conn = sqlite3.connect("./Chinook.db")
print("Opend database successfully")

Opend database successfully


In [84]:
def get_table_names(conn):
    """Return a list of table names."""
    table_names = []
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type = 'table';")
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names

def get_column_names(conn, table_name):
    """Return a list of column names."""
    column_names = []
    columns = conn.execute(f"PRAGMA table_info('{table_name}');").fetchall()
    for col in columns:
        column_names.append(col[1])
    return column_names

def get_database_info(conn):
    """Return a list of dicts containing the table name and colums for each table in the database."""
    table_dicts = []
    for table_name in get_table_names(conn):
        columns_names = get_column_names(conn, table_name)
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts

In [85]:
database_schema_dict = get_database_info(conn)
database_schema_string = "\n".join(
    [
        f"Table: {table['table_name']}\nColumns:{', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)

In [86]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "ask_database",
            "description": "Use this function to answer user questions about music. Input should be a fully formed SQL query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": f"""
                                SQL query extracting info to answer the user's question.
                                SQL should be written using this database schema:
                                {database_schema_string}
                                The query should be returned in plain text, not in JSON.
                                """,
                    }
                },
                "required": ["query"],
            },
        }
    }
]

In [94]:
def ask_database(conn, query):
    """Function to query SQLite database with a provided SQL query."""
    try:
        results = str(conn.execute(query).fetchall())
    except Exception as e:
        results = f"query failed with error {e}"
    return results

def execute_function_call(message):
    if message.tool_calls[0].function.name == "ask_database":
        query = json.loads(message.tool_calls[0].function.arguments)["query"]
        results = ask_database(conn, query)
    else:
        results = f"Error: function {message.tool_calls[0].function.name} does not exist"
    return results


In [103]:
messages = []
messages.append({"role": "system", "content": "Answer user questions by generating SQL queries against the Chinook Music Database."})
messages.append({"role": "user", "content": "Hi, who are the top 5 artists by number of tracks?"})
chat_response = chat_completion_request(messages, tools)
# 먼저 prompt 를 이해함. 
# 이후 요청사항에 대답하려면 tool calling 을 해야하는지 infer함 (여기선 multiple tool 에서 어떤 tool selection 을 할지가 아니라, tool calling YES/NO 에 대한 infer임)
# 이때 given 된 tool 의 형식을 참고함.
# tool.json 에 적혀있는 description 을 이해함
# tool.json 적혀있는 description 과 prompt 를 비교해보고, tool calling 을 해야할지 infer함
# tool call 을 한다면, tool call 을 어떻게 할지가 json 파일에 나와있음. 

# get_table_name, get_column_name => get_database_info => database_schema_dict => database_schema_string 을 그대로 따라하는게 아니라, 이건 필요한 데이터를 만드는 거고 (이걸 사용할거다 라고)

### prompt 의 요청사항을 해결하기 위해 function 을 쓸건데, 그 function 의 argument 를 어떻게 넣을건지를 infer 해주는 것이 포인트!!!
### json 파일의 역할은 tool argument지정, description으로, 
### 우리가 사용하고자 하는 api 에 들어가야 되는 형식을 지정해줌. (API argument 와 GPT generated response 를 연결해줌)
assistant_message = chat_response.choices[0].message
assistant_message.content = str(assistant_message.tool_calls[0].function) # argument generated by model (gpt) # Model infer about the required code
messages.append({"role": assistant_message.role, "content": assistant_message.content})
if assistant_message.tool_calls:
    results = execute_function_call(assistant_message)
    messages.append({"role": "function", "tool_call_id": assistant_message.tool_calls[0].id, "name": assistant_message.tool_calls[0].function.name, "content": results})
pretty_print_conversation(messages)

system: Answer user questions by generating SQL queries against the Chinook Music Database.

user: Hi, who are the top 5 artists by number of tracks?

assistant: Function(arguments='{\n  "query": "SELECT Artist.Name, COUNT(*) AS TrackCount FROM Artist INNER JOIN Album ON Artist.ArtistId = Album.ArtistId INNER JOIN Track ON Album.AlbumId = Track.AlbumId GROUP BY Artist.ArtistId ORDER BY TrackCount DESC LIMIT 5"\n}', name='ask_database')

function (ask_database): query failed with error no such table: Artist



In [111]:
messages.append({"role": "user", "content": "What is the name of the album with the most tracks?"})
chat_response = chat_completion_request(messages, tools)
assistant_message = chat_response.choices[0].message
assistant_message.content = str(assistant_message.tool_calls[0].function)
messages.append({"role": assistant_message.role, "content": assistant_message.content})
if assistant_message.tool_calls:
    results = execute_function_call(assistant_message)
    messages.append({"role": "function", "tool_call_id": assistant_message.tool_calls[0].id, "name": assistant_message.tool_calls[0].function.name, "content": results})
pretty_print_conversation(messages)

system: Answer user questions by generating SQL queries against the Chinook Music Database.

user: Hi, who are the top 5 artists by number of tracks?

assistant: Function(arguments='{\n  "query": "SELECT Artist.Name, COUNT(*) AS TrackCount FROM Artist INNER JOIN Album ON Artist.ArtistId = Album.ArtistId INNER JOIN Track ON Album.AlbumId = Track.AlbumId GROUP BY Artist.ArtistId ORDER BY TrackCount DESC LIMIT 5"\n}', name='ask_database')

function (ask_database): query failed with error no such table: Artist

user: What is the name of the album with the most tracks?

assistant: Function(arguments='{\n  "query": "SELECT Album.Title, COUNT(*) AS TrackCount FROM Album INNER JOIN Track ON Album.AlbumId = Track.AlbumId GROUP BY Album.AlbumId ORDER BY TrackCount DESC LIMIT 1"\n}', name='ask_database')

function (ask_database): query failed with error no such table: Album

user: What is the name of the album with the most tracks?

assistant: Function(arguments='{\n  "query": "SELECT Title\\nFRO

### Comments

tool calling 이 생각보다 제대로 안됨

Reasoning ability 도 매우 떨어짐

특히 수학연산

model 너무 큼